In [1]:
from pyhive import presto
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import numpy as np
import time

from datetime import datetime, timedelta

import seaborn as sns

conn = presto.connect(
    host='presto.processing.yoda.run', 
    ## presto.processing.yoda.run
    ## bi-presto.serving.data.production.internal
    port=80,
    username='manoj.ravirajan@rapido.bike'
)

In [15]:
start_date = '20221226'
end_date = '20230219'

'20220801' to '20220828' 
'20220829' to '20221030'
'20221031' to '20221225'
'20221226' to '20230219'

In [16]:
query = f"""
 
with base as (
    select 
        date_format(date_parse(yyyymmdd, '%Y%m%d'), '%Y-%m-%d') order_date, 
        date_format(DATE_TRUNC('week', date_parse(yyyymmdd, '%Y%m%d')), '%Y-%m-%d') order_week, 
        city_name, order_type, order_id, customer_id,captain_id,
        distance_final_distance,
        service_obj_service_name service,
        spd_fraud_flag,ride_time,ride_distance,
        amount,amount_breakup_base_fare_total,actual_price
        
    from orders.order_logs_snapshot
    where order_status = 'dropped' --and event_type = 'dropped'
        and yyyymmdd BETWEEN '{start_date}' and '{end_date}'
        and service_obj_service_name = 'Link'
),

segment as (
select * 
from 
(
select 
    date_format(date_parse(yyyymmdd, '%Y%m%d'), '%Y-%m-%d') order_date, 
    date_format(DATE_TRUNC('week', date_parse(yyyymmdd, '%Y%m%d')), '%Y-%m-%d') order_wk, 
    captain_id, 
    case 
    when poc_segment like 'HP_D%' then '3. HP Daily'
    when poc_segment like 'HP_I%' then '4. HP Intra/Inter'

    when poc_segment like 'MP_D%' then '5. MP Daily'
    when poc_segment like 'MP_I%' then '6. MP Intra/Inter'

    when poc_segment like 'LP_D%' then '7. LP Daily'
    when poc_segment like 'LP_I%' then '8. LP Intra/Inter'
    
    when poc_segment like 'UHP_D%' then '1. UHP Daily'
    when poc_segment like 'UHP_I%' then '2. UHP Intra/Inter'
    
    end segment,
    row_number() over(partition by date_format(DATE_TRUNC('week', date_parse(yyyymmdd, '%Y%m%d')), '%Y-%m-%d'), captain_id order by yyyymmdd desc) seq_no
from datasets.captain_metrics_segments
where
    yyyymmdd BETWEEN '{start_date}' and '{end_date}'
    and poc_segment is not null 
    -- and day_of_week(cast (date_parse(yyyymmdd, '%Y%m%d') as date)) = 7
    and captain_id in (select distinct captain_id from base)
 ) where seq_no = 1
),

total_spd_ride as (

select 
    order_week,
    service,
    count(distinct order_id) total_orders,
    count(distinct case when spd_fraud_flag = true then order_id end) as total_spd_orders,
    count(distinct case when ride_distance <= 5.0 and spd_fraud_flag = true then order_id end) as short_spd_orders,
    count(distinct case when ride_distance > 5.0 and spd_fraud_flag = true then order_id end) as long_spd_orders
from base
group by 1,2
),

spd_ride as (
select 
    order_week,
    service,
    coalesce(segment,'0. New') segment,
    count(distinct base.customer_id) uniq_customer,
    count(distinct base.captain_id) uniq_captain,
    count(distinct order_id) as total_orders,
    count(distinct case when spd_fraud_flag = true then order_id end) as total_spd_orders,
    count(distinct case when spd_fraud_flag = true then base.customer_id end) spd_uniq_customer,
    count(distinct case when spd_fraud_flag = true then base.customer_id end) spd_uniq_captain,
    count(distinct case when ride_distance <= 5.0 and spd_fraud_flag = true then order_id end) as short_spd_orders,
    count(distinct case when ride_distance > 5.0 and spd_fraud_flag = true then order_id end) as long_spd_orders,
    sum( case when spd_fraud_flag = true then amount_breakup_base_fare_total end) as actual_fare,
    sum( case when spd_fraud_flag = true then actual_price end) as estimated_fare,
    sum( case when spd_fraud_flag = true then 0.2*actual_price end) as expected_commission
from base
left join segment s on order_week = order_wk and s.captain_id = base.captain_id
group by 1,2,3
)

select 
    a.order_week,
    -- a.service,
    a.segment,
    a.total_orders,
    a.total_spd_orders,
    try(a.total_spd_orders*100.00/b.total_spd_orders) "spd_orders_%",
    a.expected_commission,
    a.short_spd_orders,
    a.long_spd_orders,
    -- try(a.short_spd_orders*100.00/b.total_spd_orders) short_spd_distr,
    -- try(a.long_spd_orders*100.00/b.total_spd_orders) long_spd_distr,
    try(a.short_spd_orders*100.00/b.short_spd_orders) "short_spd_%",
    try(a.long_spd_orders*100.00/b.long_spd_orders) "long_spd_%" ,
    a.actual_fare,
    a.estimated_fare,
    a.uniq_captain,
    a.spd_uniq_captain,
    a.uniq_customer,
    a.spd_uniq_customer
    
from spd_ride a
left join total_spd_ride b on a.order_week = b.order_week and a.service = b.service

order by 1 desc,2


"""

In [17]:
df_code = pd.read_sql(query, conn)
df_code 

,order_week,segment,total_orders,total_spd_orders,spd_orders_%,expected_commission,short_spd_orders,long_spd_orders,short_spd_%,long_spd_%,actual_fare,estimated_fare,uniq_captain,spd_uniq_captain,uniq_customer,spd_uniq_customer
0,2023-02-13,0. New,8387,1201,1.54,7270.6,675,526,1.94,1.22,15463.5,36353.0,3528,1156,8263,1156
1,2023-02-13,1. UHP Daily,520738,4165,5.35,30325.2,2058,2107,5.93,4.89,51125.5,151626.0,7378,4117,413525,4117
2,2023-02-13,2. UHP Intra/Inter,17987,198,0.25,1167.6,103,95,0.30,0.22,2473.0,5838.0,851,198,17437,198
3,2023-02-13,3. HP Daily,930076,11769,15.12,76948.2,5160,6609,14.86,15.33,141798.5,384741.0,23008,11602,697971,11602
4,2023-02-13,4. HP Intra/Inter,71458,874,1.12,5647.0,410,464,1.18,1.08,10597.0,28235.0,4833,871,68835,871
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,2022-12-26,4. HP Intra/Inter,46897,577,0.85,3097.2,304,273,0.92,0.80,6719.0,15486.0,3491,575,45585,575
68,2022-12-26,5. MP Daily,1491908,26613,39.42,154252.4,12916,13697,38.90,39.92,318388.0,771262.0,83161,25994,1012185,25994
69,2022-12-26,6. MP Intra/Inter,171588,3234,4.79,20404.4,1745,1489,5.26,4.34,38411.0,102022.0,25543,3188,160089,3188
70,2022-12-26,7. LP Daily,544524,18319,27.14,108232.6,8579,9740,25.84,28.39,212755.5,541163.0,70044,17979,436718,17979


In [6]:
df1 = df_code
df1.head()

,order_week,segment,total_orders,total_spd_orders,spd_orders_%,expected_commission,short_spd_orders,long_spd_orders,short_spd_%,long_spd_%,actual_fare,estimated_fare,uniq_captain,spd_uniq_captain,uniq_customer,spd_uniq_customer
0,2022-08-22,0. New,512641,10634,17.47,63742.0,6972,3662,21.01,13.23,125432.5,318710.0,41488,9968,272234,9968
1,2022-08-22,1. UHP Daily,371293,2642,4.34,17174.0,1544,1098,4.65,3.97,33597.0,85870.0,5013,2594,289056,2594
2,2022-08-22,2. UHP Intra/Inter,11844,103,0.17,665.6,76,27,0.23,0.10,1263.5,3328.0,536,103,11149,103
3,2022-08-22,3. HP Daily,771005,8187,13.45,44528.6,4392,3795,13.24,13.71,102732.5,222643.0,18583,8015,557639,8015
4,2022-08-22,4. HP Intra/Inter,57544,641,1.05,3155.6,365,276,1.10,1.00,7823.5,15778.0,3978,634,55163,634


In [10]:
df2 = df_code
df2.head()

,order_week,segment,total_orders,total_spd_orders,spd_orders_%,expected_commission,short_spd_orders,long_spd_orders,short_spd_%,long_spd_%,actual_fare,estimated_fare,uniq_captain,spd_uniq_captain,uniq_customer,spd_uniq_customer
0,2022-10-24,0. New,945,887,1.71,4933.0,562,325,2.23,1.22,10616.5,24665.0,741,819,876,819
1,2022-10-24,1. UHP Daily,296792,2208,4.25,17153.4,1287,921,5.11,3.44,28769.0,85767.0,5497,2181,250096,2181
2,2022-10-24,2. UHP Intra/Inter,10359,83,0.16,417.4,52,31,0.21,0.12,1117.0,2087.0,583,82,9807,82
3,2022-10-24,3. HP Daily,609433,7206,13.87,41779.6,3617,3589,14.35,13.42,88907.5,208898.0,18470,7089,475724,7089
4,2022-10-24,4. HP Intra/Inter,42675,540,1.04,3094.2,298,242,1.18,0.91,6704.5,15471.0,3464,533,41272,533


In [14]:
df3 = df_code
df3.head()

,order_week,segment,total_orders,total_spd_orders,spd_orders_%,expected_commission,short_spd_orders,long_spd_orders,short_spd_%,long_spd_%,actual_fare,estimated_fare,uniq_captain,spd_uniq_captain,uniq_customer,spd_uniq_customer
0,2022-12-19,0. New,1626,1136,1.64,6867.2,676,460,1.95,1.33,13859.0,34336.0,1192,1064,1546,1064
1,2022-12-19,1. UHP Daily,470625,3440,4.98,19835.4,2042,1398,5.89,4.06,42860.5,99177.0,6753,3244,367169,3244
2,2022-12-19,2. UHP Intra/Inter,12217,128,0.19,663.6,92,36,0.27,0.10,1547.0,3318.0,593,128,11657,128
3,2022-12-19,3. HP Daily,962465,10882,15.74,59596.2,5351,5531,15.43,16.05,129031.0,297981.0,23536,10639,703470,10639
4,2022-12-19,4. HP Intra/Inter,50573,687,0.99,3716.0,382,305,1.10,0.89,7981.5,18580.0,3754,682,49056,682


In [18]:
df4 = df_code
df4.head()

,order_week,segment,total_orders,total_spd_orders,spd_orders_%,expected_commission,short_spd_orders,long_spd_orders,short_spd_%,long_spd_%,actual_fare,estimated_fare,uniq_captain,spd_uniq_captain,uniq_customer,spd_uniq_customer
0,2023-02-13,0. New,8387,1201,1.54,7270.6,675,526,1.94,1.22,15463.5,36353.0,3528,1156,8263,1156
1,2023-02-13,1. UHP Daily,520738,4165,5.35,30325.2,2058,2107,5.93,4.89,51125.5,151626.0,7378,4117,413525,4117
2,2023-02-13,2. UHP Intra/Inter,17987,198,0.25,1167.6,103,95,0.30,0.22,2473.0,5838.0,851,198,17437,198
3,2023-02-13,3. HP Daily,930076,11769,15.12,76948.2,5160,6609,14.86,15.33,141798.5,384741.0,23008,11602,697971,11602
4,2023-02-13,4. HP Intra/Inter,71458,874,1.12,5647.0,410,464,1.18,1.08,10597.0,28235.0,4833,871,68835,871


In [19]:
frames = [df1,df2,df3,df4]
results = pd.concat(frames)

In [20]:
results

,order_week,segment,total_orders,total_spd_orders,spd_orders_%,expected_commission,short_spd_orders,long_spd_orders,short_spd_%,long_spd_%,actual_fare,estimated_fare,uniq_captain,spd_uniq_captain,uniq_customer,spd_uniq_customer
0,2022-08-22,0. New,512641,10634,17.47,63742.0,6972,3662,21.01,13.23,125432.5,318710.0,41488,9968,272234,9968
1,2022-08-22,1. UHP Daily,371293,2642,4.34,17174.0,1544,1098,4.65,3.97,33597.0,85870.0,5013,2594,289056,2594
2,2022-08-22,2. UHP Intra/Inter,11844,103,0.17,665.6,76,27,0.23,0.10,1263.5,3328.0,536,103,11149,103
3,2022-08-22,3. HP Daily,771005,8187,13.45,44528.6,4392,3795,13.24,13.71,102732.5,222643.0,18583,8015,557639,8015
4,2022-08-22,4. HP Intra/Inter,57544,641,1.05,3155.6,365,276,1.10,1.00,7823.5,15778.0,3978,634,55163,634
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,2022-12-26,4. HP Intra/Inter,46897,577,0.85,3097.2,304,273,0.92,0.80,6719.0,15486.0,3491,575,45585,575
68,2022-12-26,5. MP Daily,1491908,26613,39.42,154252.4,12916,13697,38.90,39.92,318388.0,771262.0,83161,25994,1012185,25994
69,2022-12-26,6. MP Intra/Inter,171588,3234,4.79,20404.4,1745,1489,5.26,4.34,38411.0,102022.0,25543,3188,160089,3188
70,2022-12-26,7. LP Daily,544524,18319,27.14,108232.6,8579,9740,25.84,28.39,212755.5,541163.0,70044,17979,436718,17979


In [21]:
results.to_csv("spd_link.csv", index = False)